In [1]:
import numpy as numpy
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import naive_bayes

In [2]:
!wget https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip -o sms+spam+collection.zip
!unzip sms+spam+collection.zip
!rm sms+spam+collection.zip*

Archive:  sms+spam+collection.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of sms+spam+collection.zip or
        sms+spam+collection.zip.zip, and cannot find sms+spam+collection.zip.ZIP, period.


In [3]:
def vectorize_dataset(filename):
    messages = []
    labels = []
    with open(filename, 'r') as f:
        for line in f:
            label, text = line.split(maxsplit=1)
            messages.append(text.lower())
            labels.append(int(label=="spam"))

    vectorizer = CountVectorizer(stop_words='english')
    X = vectorizer.fit_transform(messages)
    keywords = vectorizer.get_feature_names_out()
    
    df = pd.DataFrame(data=X.todense(), columns=keywords)
    df.insert(loc=0, column="#label#", value=labels)
    return df, messages

In [4]:
df, messages = vectorize_dataset(filename='SMSSpamCollection')
print(f"Number of words = {len(df.columns)}")
df.head()

Number of words = 8445


,#label#,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,...,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,èn,ú1,〨ud
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
keywords = df.columns[1:]
non_english_words = [w for w in keywords if not w.isalpha()]
print(non_english_words)
df.drop(non_english_words, axis=1, inplace=True)

['00', '000', '000pes', '008704050406', '0089', '0121', '01223585236', '01223585334', '0125698789', '02', '0207', '02072069400', '02073162414', '02085076972', '021', '03', '04', '0430', '05', '050703', '0578', '06', '07', '07008009200', '07046744435', '07090201529', '07090298926', '07099833605', '07123456789', '0721072', '07732584351', '07734396839', '07742676969', '07753741225', '0776xxxxxxx', '07781482378', '07786200117', '077xxx', '078', '07801543489', '07808', '07808247860', '07808726822', '07815296484', '07821230901', '078498', '07880867867', '0789xxxxxxx', '07946746291', '0796xxxxxx', '07973788240', '07xxxxxxxxx', '08', '0800', '08000407165', '08000776320', '08000839402', '08000930705', '08000938767', '08001950382', '08002888812', '08002986030', '08002986906', '08002988890', '08006344447', '0808', '08081263000', '08081560665', '0825', '083', '0844', '08448350055', '08448714184', '0845', '08450542832', '08452810071', '08452810073', '08452810075over18', '0870', '08700435505150p', '

In [6]:
df.duplicated().any(), df.duplicated().sum()

(True, 565)

In [7]:
df.drop_duplicates(inplace=True)

In [8]:
df["#label#"].value_counts(normalize=True)

#label#
0    0.887003
1    0.112997
Name: proportion, dtype: float64

In [9]:
X = df.iloc[:,1:].values
y = df['#label#'].values

Cross Validation

In [10]:
model = naive_bayes.NaiveBayesFromScratch(distribution="bernoulli")
scores = naive_bayes.cross_validation(model, X, y, 10)
scores.mean()

0.9763999999999999

In [ ]:
# model = naive_bayes.NaiveBayesFromScratch(distribution="gaussian")
# scores = naive_bayes.cross_validation(model, X, y, 10)
# scores.mean()

/home/malick/Desktop/machine-learning-grind/01-machine-learning/01-stats/04-naives-bayes/naive_bayes.py:69: RuntimeWarning: invalid value encountered in scalar divide
  return np.exp(-(xi-mu)**2 / (2*sigma**2)) / (np.sqrt(2*np.pi) * sigma)
/home/malick/Desktop/machine-learning-grind/01-machine-learning/01-stats/04-naives-bayes/naive_bayes.py:69: RuntimeWarning: divide by zero encountered in scalar divide
  return np.exp(-(xi-mu)**2 / (2*sigma**2)) / (np.sqrt(2*np.pi) * sigma)
